#### MST: IASD, NLP: LAB 1
this notebook describes the steps of scraping an arabic sports website for articles, inserting data in MongoDB, establishing of NLP pipeline and using different NLP technics such us stemming, lemmetization, PoS and NER.
this work is done by **Aymen Ait Ahmed**

In [2]:
from bs4 import BeautifulSoup
import requests as rs

### Scraping www.almayadeen.net/all-sports using BeautifulSoup library

In [3]:
url = "https://www.almayadeen.net/all-sports"

soup = BeautifulSoup(rs.get(url).content.decode('UTF-8'))

In [4]:
articles = []
for i in soup.find_all('div', {'class', 'grid-item'}):
    try:
        article_html = i.find('div', {'class', 'grid-title fixed-height'})
        temp = str(article_html.a['href'].encode('utf8')).replace(r'\x', '%').replace('/sports',
                                                                                      '').upper().removeprefix(
            "B'").removesuffix("'")
        article_link = 'https://www.almayadeen.net/sports' + temp
        article_title = article_html.a.h4.text
        temp_soup = BeautifulSoup(rs.get(article_link).text)
        article_content = ''.join(
            [i.text for i in temp_soup.find('div', {'class', 'p-content'}).find_all('p', recursive=False)])
        article = {'link': article_link, 'title': article_title, 'content': article_content}
        articles.append(article)
    except:
        continue

In [5]:
len(articles)

12

### Inserting data in MongoDB

In [65]:
!pip install pymongo


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/6b/38/11db8be24e20b0885182aaa3839db2eb2be51f506a222538762e1b3723f5/pymongo-4.6.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/87/a1/8c5287991ddb8d3e4662f71356d9656d91ab3a36618c3dd11b280df0d255/dnspython-2.6.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.9 kB ? eta -:--:--
   --- ----------------------------------- 41.0/472.9 kB 960.0 kB/s eta 0:00:01
   ----- --------------------------------- 61.4/472.9 kB 648.1 kB/s eta 0:00:01
   --------- ------------------------------ 112.6/472.9 kB 1.1 MB/s eta 0:00:01
   --------- ---------------------------- 122.9/472.9 kB 654.9 kB/s eta 0:00:01
   ------------------- -------------------- 235.5/472.9 kB 1.0 MB/s eta 0:00:01
   ------------------------ --------------- 286.7/472.9 kB 1.0 MB/s eta 0:00:01
   -------------------

In [66]:
import pymongo

mongo_client = pymongo.MongoClient('mongodb://127.0.0.1:27017')


In [67]:
db = mongo_client['articles']
db_collection = db.sportArticles

In [69]:
db_collection.insert_many(articles)

InsertManyResult([ObjectId('660c63115408d82d214bf57b'), ObjectId('660c63115408d82d214bf57c'), ObjectId('660c63115408d82d214bf57d'), ObjectId('660c63115408d82d214bf57e'), ObjectId('660c63115408d82d214bf57f'), ObjectId('660c63115408d82d214bf580'), ObjectId('660c63115408d82d214bf581'), ObjectId('660c63115408d82d214bf582'), ObjectId('660c63115408d82d214bf583'), ObjectId('660c63115408d82d214bf584'), ObjectId('660c63115408d82d214bf585'), ObjectId('660c63115408d82d214bf586')], acknowledged=True)

### Establishment of NLP Pipeline

In [70]:
!pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/a8/01/18232f93672c1d530834e2e0568a80eaab1df12d67ae499b1762ab462b5c/regex-2023.12.25-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/2a/14/e75e


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...


True

#### Tokenization

In [6]:
from nltk.tokenize import word_tokenize

tokenized_titles = []
tokenized_contents = []
len_before = 0
for article in articles:
    tokenized_title = word_tokenize(article['title'])
    len_before += len(tokenized_title)
    tokenized_titles.append(tokenized_title)
    tokenized_content = word_tokenize(article['content'])
    len_before += len(tokenized_content)
    tokenized_contents.append(tokenized_content)


In [88]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

#### StopWords removal

In [7]:
from nltk.corpus import stopwords

stopwords = set(stopwords.words("arabic"))
filtred_titles = [word for title in tokenized_titles for word in title if not word in stopwords]
filtred_content = [word for title in tokenized_contents for word in title if not word in stopwords]
filtred_collection = filtred_titles + filtred_content
print('before filtration: ',len_before)
print('after filtration: ',len(filtred_collection))

before filtration:  1908
after filtration:  1554


#### Stemming using ISRIStemmer

In [8]:
from nltk.stem import ISRIStemmer

stemmer = ISRIStemmer()
stemmed_words = [stemmer.stem(word) for word in filtred_collection]
for i in range(10):
    print(f'went from {filtred_collection[i]} to {stemmed_words[i]}')

went from ميسي to يسي
went from يواصل to يصل
went from الغياب to غيب
went from وإنتر to إنتر
went from ميامي to يمي
went from يخسر to خسر
went from باريس to ارس
went from سان to سان
went from جيرمان to جيرم
went from يعود to يعد


#### Lemmetization using Farasa API
Farasa is the state-of-the-art full-stack package to deal with Arabic Language Processing. It has been developed by **Arabic Language Technologies Group** at **Qatar Computing Research Institute (QCRI)**. It has a RESTful Web API that you can use through your favorable programming language.

In [ ]:
import json
import requests
url = 'https://farasa.qcri.org/webapi/lemmatization/'
t = 0
api_key = "vbnkEBqAmuLmFEjfqd"

In [10]:
payload = {'text': ' '.join(filtred_collection), 'api_key': api_key}
data = requests.post(url, data=payload)
text = json.loads(data.text)
text

{'text': ['ميس',
  'واصل',
  'غياب',
  'إنتر',
  'ميامي',
  'خسر',
  'باريس',
  'سان',
  'جيرمان',
  'عاد',
  'نهائي',
  'كأس',
  'فرنسا',
  'واجه',
  'ليون',
  'أرجنتين',
  'حافظ',
  'تصنيف',
  '`',
  '`',
  'فيفا',
  "'",
  "'",
  'بطولة',
  'وصل',
  ':',
  'حكمة',
  'هزم',
  'رياضي',
  'فرض',
  'مباراة',
  'فاصل',
  'مانشستر',
  'سيتي',
  'اكتسح',
  'أستون',
  'فيلا',
  'رباعي',
  'فيورنتينا',
  'هزم',
  'أتالانتا',
  'ذهاب',
  'نصف',
  'نهائي',
  'كأس',
  'إيطاليا',
  '(',
  'فيديو',
  ')',
  "'",
  "'",
  'بريميرليغ',
  "'",
  "'",
  ':',
  'أرسنال',
  'تصدر',
  'موقت',
  'فوز',
  'لوت',
  'تاون',
  'اعتقال',
  'مطار',
  'مدريد',
  '.',
  '.',
  'إطلاق',
  'سراح',
  'لويس',
  'روبياليس',
  'مانشستر',
  'يونايتد',
  'أعلن',
  'إصابة',
  'ليساندرو',
  'مارتينيز',
  'ليندلوف',
  'اعتقال',
  'رئيس',
  'اتحاد',
  'إسباني',
  'سابق',
  'لويس',
  'روبياليس',
  'فار',
  'أصاب',
  'ارتجاج',
  'دماغ',
  '.',
  '.',
  'احذر',
  'ضربة',
  'رأسي',
  'روبياليس',
  'خرج',
  'صمت',
  '.',
  '.',


In [11]:
lemmetized_words = text["text"]

for i in range(10):
    print(f'went from {filtred_collection[i]} to {lemmetized_words[i]}')

went from ميسي to ميس
went from يواصل to واصل
went from الغياب to غياب
went from وإنتر to إنتر
went from ميامي to ميامي
went from يخسر to خسر
went from باريس to باريس
went from سان to سان
went from جيرمان to جيرمان
went from يعود to عاد


#### Comparison between Stemming and Lemmetization

In [41]:
import pandas as pd
ls = ['(', '،', ')', '.', '/', ':', '؟', 'ب', '`', 'ل', "'", '؛'] 
lemmetized_words_f = [i for i in lemmetized_words if not i in ls]
stemmed_words_f = [i for i in stemmed_words if not i in ls]
filtred_collection_f = [i for i in filtred_collection if not i in ls]

1507,  1504  1507


In [44]:
df = pd.DataFrame(data={'original':filtred_collection_f[:len(lemmetized_words_f)],'stemmed':stemmed_words_f[:len(lemmetized_words_f)],'lemmetized':lemmetized_words_f})
df

,original,stemmed,lemmetized
0,ميسي,يسي,ميس
1,يواصل,يصل,واصل
2,الغياب,غيب,غياب
3,وإنتر,إنتر,إنتر
4,ميامي,يمي,ميامي
...,...,...,...
1499,للتحقيق,حقق,تعلق
1500,قضايا,قضا,سوء
1501,فساد,فسد,إدارة
1502,تتعلق,علق,غسل


#### PoS tagging using Farasa PoS Tagger

In [45]:
PoS_url = 'https://farasa.qcri.org/webapi/pos/'
payload = {'text': " ".join(lemmetized_words_f), 'api_key': api_key}
data = requests.post(PoS_url, data=payload)
result = json.loads(data.text)
result

{'text': [{'surface': 'S', 'POS': 'S', 'num': '', 'position': 'B'},
  {'surface': 'ميس', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'واصل', 'POS': 'V', 'num': '', 'position': 'B'},
  {'surface': 'غياب', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'إنتر', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'ميامي', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'خسر', 'POS': 'V', 'num': '', 'position': 'B'},
  {'surface': 'باريس', 'POS': 'NOUN', 'num': 'MP', 'position': 'B'},
  {'surface': 'سان', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'جيرمان', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'عاد', 'POS': 'V', 'num': '', 'position': 'B'},
  {'surface': 'نهائي', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'كأس', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'فرنسا', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'},
  {'surface': 'واجه', 'POS': 'V', 'num': '', 'position': 'B'},

In [51]:
pos_df = pd.DataFrame({"word":[i['surface'] for i in result["text"]],'PoS_Tag':[i['POS'] for i in result["text"]]})
pos_df

,word,PoS_Tag
0,S,S
1,ميس,NOUN
2,واصل,V
3,غياب,NOUN
4,إنتر,NOUN
...,...,...
1507,سوء,NOUN
1508,إدار +ة,NOUN+NSUFF
1509,غسل,NOUN
1510,مال,NOUN


#### Named Entity Recognizer Using Farasa

In [52]:
NER_url = 'https://farasa.qcri.org/webapi/ner/'
payload = {'text': " ".join(lemmetized_words_f), 'api_key': api_key}
data = requests.post(NER_url, data=payload)
result = json.loads(data.text)
result

{'text': ['ميس/O',
  'واصل/O',
  'غياب/O',
  'إنتر/B-ORG',
  'ميامي/I-ORG',
  'خسر/O',
  'باريس/B-ORG',
  'سان/I-ORG',
  'جيرمان/I-ORG',
  'عاد/O',
  'نهائي/O',
  'كأس/O',
  'فرنسا/B-LOC',
  'واجه/O',
  'ليون/B-ORG',
  'أرجنتين/I-ORG',
  'حافظ/B-PERS',
  'تصنيف/O',
  'فيفا/O',
  'بطولة/O',
  'وصل/O',
  'حكمة/O',
  'هزم/O',
  'رياضي/O',
  'فرض/O',
  'مباراة/O',
  'فاصل/O',
  'مانشستر/B-ORG',
  'سيتي/I-ORG',
  'اكتسح/O',
  'أستون/B-ORG',
  'فيلا/I-ORG',
  'رباعي/I-ORG',
  'فيورنتينا/B-ORG',
  'هزم/O',
  'أتالانتا/O',
  'ذهاب/O',
  'نصف/O',
  'نهائي/O',
  'كأس/O',
  'إيطاليا/B-LOC',
  'فيديو/O',
  'بريميرليغ/O',
  'أرسنال/B-ORG',
  'تصدر/O',
  'موقت/O',
  'فوز/O',
  'لوت/O',
  'تاون/O',
  'اعتقال/O',
  'مطار/O',
  'مدريد/B-LOC',
  'إطلاق/O',
  'سراح/O',
  'لويس/O',
  'روبياليس/O',
  'مانشستر/B-ORG',
  'يونايتد/I-ORG',
  'أعلن/O',
  'إصابة/O',
  'ليساندرو/B-PERS',
  'مارتينيز/I-PERS',
  'ليندلوف/O',
  'اعتقال/O',
  'رئيس/O',
  'اتحاد/O',
  'إسباني/O',
  'سابق/O',
  'لويس/B-PERS',
  'روبيال

In [53]:
ner_df = pd.DataFrame({"word":[i.split(sep='/')[0] for i in result["text"]],'NER_Tag':[i.split(sep='/')[1] for i in result["text"]]})
ner_df

,word,NER_Tag
0,ميس,O
1,واصل,O
2,غياب,O
3,إنتر,B-ORG
4,ميامي,I-ORG
...,...,...
1499,تعلق,O
1500,سوء,O
1501,إدارة,O
1502,غسل,O


### Conclution
Throughout this NoteBook i have learned to use BeautifulSoup to scrap Data from a website, deal with the encoding of arabic, make simple CRUD operations using MongoDB in Python, establishing a good NLP pipeline that can deal with Arabic, research how the NLP is used to treat Arabic with different technics.
to sum up everything that has been said the technics and researches that where made in this domain, although they are still not enough but giving the difficulty of the language and the lack of good and financed scientific research in this field its understandable, and we hope to continue making it better